# Friends Data Scraper
* Created by: Nirvan S P Theethira
* Date: 12/01/2019
* Purpose:  CSCI 5266 Fall Group Project: Mimic
* This files contains the code to extract charachter dialogue data from friends scripts.

In [1]:
import numpy as np
import os
import pandas as pd
import re
import pickle

## Function to read a csv script file
* Reads a script and extract a particular charachters `person` response to a dialogue. The dialogue the charachter respons to is also noted down. The inputs and the outputs for a charachter are returned.

In [2]:
def readFile(lines, person):
    scene = ''
    preChar = ''
    preScene = ''
    dialogue = ''
    # dataF = {'scene':[],'person':[], 'dialogue':[]}
    dataF = []
    dataRaw = ''
    prevDialogue = ''
    inputs = []
    outputs = []
    for line in lines:
        line = re.sub('\n','',line)
        if re.search(r"^\[[\w\W]+\]$", re.sub('\n','',line)):
            scene = line
        if re.search('^[a-zA-Z]+:',line) and len(scene)>0:
            charachter = re.findall('^[a-zA-Z]+:',line)[0]
            charachter = re.sub(':','',charachter).upper()
            if preChar!=charachter and len(preChar)!=0:
                if preChar==person:
                    dialogue = re.sub('\((.*?)\)','',dialogue)
                    dialogue = re.sub("[’']","",dialogue)
                    dialogue = re.sub('^\s+','',dialogue)
                    dialogue = re.sub('\s+$','',dialogue)
                    dataRaw += prevDialogue+'\t'+dialogue+'\n'
                    inputs.append(prevDialogue)
                    outputs.append(dialogue)
                dataF.append([preChar.upper(),dialogue])
                prevDialogue = re.sub('\((.*?)\)','',dialogue)
                prevDialogue = re.sub("[’']","",prevDialogue)
                prevDialogue = re.sub('^\s+','',prevDialogue)
                prevDialogue = re.sub('\s+$','',prevDialogue)
                dialogue=''

            dialogue += re.sub('^[a-zA-Z]+:','',line)
            preChar = charachter
    return dataRaw, inputs, outputs

## Extraction list of episode files

In [3]:
episodes = []
for (dirpath, dirnames, filenames) in os.walk('transcripts_friends\\season_all\\'):
    for file in filenames:
        if '.csv' in file:
            episodes.append(dirpath+file)

## Extraction inputs and outputs for a particular `person` from all eposodes collected

In [5]:
inputs = []
outputs = []
person = 'ROSS'
for episode in episodes:
#     print(episode)
    with open(episode,encoding='utf8') as f:
        lines = f.readlines()
        _, inp, out = readFile(lines, person)
        inputs+=inp
        outputs+=out
print(inputs[0:3])
print(outputs[0:3])
print(len(inputs),len(outputs))

['Well, okay, Mikes taking a shower, which by the way theres no law against. And then were gonna grab some food, so if you want...', 'Ooh... Dr. Geller!', 'You gotta hear this, its great... Its like free porn!']
['... finally...', 'God, youre amazing... I didnt even have to ask you to call me that.', 'Weh...*sigh*']
8617 8617


## Spliting into train and test and saving collected input and output data.

In [15]:
trainTestSplit = 0.8
pickle.dump(inputs[:int(len(inputs)*trainTestSplit)], 
            open('data\\{}Data\\{}InputTrain.pkl'.format(person.lower(),person.lower()), 'wb'))
pickle.dump(inputs[int(len(inputs)*trainTestSplit):], 
            open('data\\{}Data\\{}InputTest.pkl'.format(person.lower(),person.lower()), 'wb'))
pickle.dump(outputs[:int(len(inputs)*trainTestSplit)], 
            open('data\\{}Data\\{}OutputTrain.pkl'.format(person.lower(),person.lower()), 'wb'))
pickle.dump(outputs[int(len(inputs)*trainTestSplit):], 
            open('data\\{}Data\\{}OutputTest.pkl'.format(person.lower(),person.lower()), 'wb'))